In [1]:
using CSV, DataFrames, Random, Gurobi, JuMP, Statistics, ScikitLearn, GLM

## Loading data

In [2]:
# read in data
X_train_df = CSV.read("Preprocessed/X_train_scaled.csv", DataFrame)
X_test_df = CSV.read("Preprocessed/X_test_scaled.csv", DataFrame)
Y_train_df = CSV.read("Preprocessed/y_train.csv", DataFrame)
Y_test_df = CSV.read("Preprocessed/y_test.csv", DataFrame);

In [3]:
# Check the size of the data
size(X_train_df), size(X_test_df), size(Y_train_df), size(Y_test_df)

((8818, 19), (2205, 19), (8818, 3), (2205, 3))

In [4]:
X_train_df

Row,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,members_in_household,months_current_account,months_customer,call_center_contacts,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.209025,0.0882424,-0.209025,1.02178,-0.0543805,-0.177725,-0.0770195,-1.19193,-0.708545,0.398425,-1.1002,-1.0644,1.11758,0.65647,0.222585,0.00842164,0.0111827,1.23981,0.32641
2,0.656889,0.9698,0.656889,-0.97868,-0.0543805,-0.177725,-0.0770195,0.25933,-1.04821,-1.31149,-0.201499,-0.237377,-0.158208,2.11148,1.34866,0.00842164,0.274139,-0.458193,0.699145
3,0.13734,0.480046,0.13734,1.02178,-0.0543805,-0.177725,-0.0770195,-0.673623,0.251026,1.53837,-0.426175,-0.237377,0.479688,1.38397,0.785624,-0.208086,-0.51473,0.900211,0.780277
4,-0.641982,-0.891266,-0.641982,-0.97868,-0.0543805,-0.177725,-0.0770195,1.60693,0.36844,-1.31149,1.22145,1.41666,-1.434,-0.798538,-0.903493,-0.424593,-0.251774,-1.5902,-0.867649
5,-0.815165,-0.695364,-0.815165,1.02178,-0.0543805,-0.177725,-0.0770195,-0.777284,0.160493,-0.741521,-1.99891,-1.89142,0.479688,-0.798538,-0.903493,-0.424593,-0.51473,1.01341,-0.867649
6,-2.28722,-1.08717,-2.28722,-0.97868,-0.0543805,5.62666,-0.0770195,1.91791,-1.03938,0.398425,0.397638,0.589643,-1.434,1.38397,0.222585,-0.208086,-1.04064,-1.5902,0.17651
7,1.17644,1.26365,1.17644,1.02178,-0.0543805,-0.177725,-0.0770195,1.19228,-0.332109,-1.88147,1.37124,1.41666,-1.434,-0.798538,0.222585,0.00842164,1.85188,-2.1562,0.574807
8,1.08985,1.06775,1.08985,-0.97868,-0.0543805,-0.177725,-0.0770195,-2.12488,2.54603,-0.741521,-1.02531,-1.0644,2.39337,-0.798538,1.34866,0.224929,-0.777687,2.71142,1.04257
9,0.656889,0.675947,0.656889,1.02178,-0.0543805,-0.177725,-0.0770195,0.673975,-1.20019,-0.741521,-1.69934,-1.89142,-0.796104,-0.798538,-0.903493,-0.424593,1.06301,-1.1374,-0.867649


In [5]:
# feature engineering
# X_train_df = X_train_df[!, [:age, :income, :members_in_household, :loan_accounts]]
# X_test_df = X_test_df[!, [:age, :income, :members_in_household, :loan_accounts]]

In [6]:
# Define products
products = ["Savings", "Mortgage", "Pension"]

# Define unit value of each product
productValue = [200, 300, 400]
value_per_product = Dict(products[i] => productValue[i] for i in 1:length(products))

# Define total available budget
availableBudget = 25000 # /(11023/1000)

# For each channel, cost of making a marketing action and success factor
# Define the data
data = [("gift", 20.0, 0.20), 
        ("newsletter", 15.0, 0.05), 
        ("seminar", 23.0, 0.30)]

# Create a DataFrame
channels = DataFrame(channel = [x[1] for x in data],
                     cost = [x[2] for x in data],
                     success_prob = [x[3] for x in data]);

In [7]:
# get input data
I = size(X_train_df, 1)
J = size(products, 1)
K = size(channels, 1)
F = size(X_train_df, 2)
X_train = Matrix(X_train_df)
X_test = Matrix(X_test_df)
Y_train = Matrix(Y_train_df)
Y_test = Matrix(Y_test_df)
p = productValue
s = channels[!, :success_prob]
c = channels[!, :cost]
B = availableBudget;

# Predicting

### Random Forest

In [8]:
# Train Random Forest using IAI using the training set
grid_rf_1 = IAI.GridSearch(
    IAI.RandomForestClassifier(
        random_seed=123,
        max_categoric_levels_before_warning=15,
        criterion =:gini,
        num_trees = 100,
        cp = 0.001
        ),
        max_depth=1:6
    )        
IAI.fit_cv!(grid_rf_1,X_train, Y_train[:, 1], validation_criterion = :auc, n_folds=5);
lnr_rf_1 = IAI.get_learner(grid_rf_1)

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ nothing nothing:nothing
┌ Warning: Interpretable AI license expires soon: 2023-12-31T00:00:00. If you need to renew, please send us the following machine ID:
│ d55e42bb8a3aa8ff0f65654e9f3b3b81ffcf09c90d5928c2a63dbd99744e8f3c
└ @ nothing nothing:nothing


Fitted RandomForestClassifier

In [9]:
# make predictions on the test set
Y_pred_rf_1 = IAI.predict(lnr_rf_1, X_test)
# compute the accuracy
accuracy_rf_1 = sum(Y_pred_rf_1 .== Y_test[:, 1]) / length(Y_test[:, 1])
# compute the AUC
auc_rf_1 = IAI.score(lnr_rf_1, X_test, Y_test[:, 1], criterion = :auc)

0.9376676355842863

In [10]:
# Train Random Forest using IAI using the training set
grid_rf_2 = IAI.GridSearch(
    IAI.RandomForestClassifier(
        random_seed=123,
        max_categoric_levels_before_warning=15,
        criterion =:gini,
        num_trees = 100,
        cp = 0.001
        ),
        max_depth=1:6
    )        
IAI.fit_cv!(grid_rf_2,X_train, Y_train[:, 2], validation_criterion = :auc, n_folds=5);
lnr_rf_2 = IAI.get_learner(grid_rf_2)

Fitted RandomForestClassifier

In [11]:
# make predictions on the test set
Y_pred_rf_2 = IAI.predict(lnr_rf_2, X_test)
# compute the accuracy
accuracy_rf_2 = sum(Y_pred_rf_2 .== Y_test[:, 2]) / length(Y_test[:, 2])
# compute the AUC
auc_rf_2 = IAI.score(lnr_rf_2, X_test, Y_test[:, 2], criterion = :auc)

0.7782625176158022

In [12]:
# Train Random Forest using IAI using the training set
grid_rf_3 = IAI.GridSearch(
    IAI.RandomForestClassifier(
        random_seed=123,
        max_categoric_levels_before_warning=15,
        criterion =:gini,
        num_trees = 100,
        cp = 0.001
        ),
        max_depth=1:6
    )        
IAI.fit_cv!(grid_rf_3,X_train, Y_train[:, 3], validation_criterion = :auc, n_folds=5);
lnr_rf_3 = IAI.get_learner(grid_rf_3)

Fitted RandomForestClassifier

In [13]:
# make predictions on the test set
Y_pred_rf_3 = IAI.predict(lnr_rf_3, X_test)
# compute the accuracy
accuracy_rf_3 = sum(Y_pred_rf_3 .== Y_test[:, 3]) / length(Y_test[:, 3])
# compute the AUC
auc_rf_3 = IAI.score(lnr_rf_3, X_test, Y_test[:, 3], criterion = :auc)

0.6861229894383495

In [14]:
# combine predictions
Y_pred_rf = hcat(Y_pred_rf_1, Y_pred_rf_2, Y_pred_rf_3)
sum(Y_pred_rf, dims=1)

1×3 Matrix{Int64}:
 193  0  102

In [15]:
# create a 200 * 3 matrix with random 0 or 1s
Random.seed!(123)
Y_pred_random = rand(0:1, 200, 3)
sum(Y_pred_random, dims=1)

1×3 Matrix{Int64}:
 106  110  116

In [16]:
sum(Matrix(Y_test_df),dims=1)

1×3 Matrix{Int64}:
 333  491  844

In [17]:
# transfer the prediction matrix to a dataframe
Y_pred_rf_df = DataFrame(Y_pred_rf, [:Savings, :Mortgage, :Pension]);

### Logistic Regression

In [18]:
# combine X_train and Y_train
XY_train_df = hcat(X_train_df, Y_train_df)

Row,age,age_youngest_child,debt_equity,gender,bad_payment,gold_card,pension_plan,household_debt_to_equity_ratio,income,members_in_household,months_current_account,months_customer,call_center_contacts,loan_accounts,number_products,number_transactions,non_worker_percentage,white_collar_percentage,rfm_score,Mortgage,Pension,Savings
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,-0.209025,0.0882424,-0.209025,1.02178,-0.0543805,-0.177725,-0.0770195,-1.19193,-0.708545,0.398425,-1.1002,-1.0644,1.11758,0.65647,0.222585,0.00842164,0.0111827,1.23981,0.32641,0,0,0
2,0.656889,0.9698,0.656889,-0.97868,-0.0543805,-0.177725,-0.0770195,0.25933,-1.04821,-1.31149,-0.201499,-0.237377,-0.158208,2.11148,1.34866,0.00842164,0.274139,-0.458193,0.699145,1,0,0
3,0.13734,0.480046,0.13734,1.02178,-0.0543805,-0.177725,-0.0770195,-0.673623,0.251026,1.53837,-0.426175,-0.237377,0.479688,1.38397,0.785624,-0.208086,-0.51473,0.900211,0.780277,0,0,0
4,-0.641982,-0.891266,-0.641982,-0.97868,-0.0543805,-0.177725,-0.0770195,1.60693,0.36844,-1.31149,1.22145,1.41666,-1.434,-0.798538,-0.903493,-0.424593,-0.251774,-1.5902,-0.867649,0,0,0
5,-0.815165,-0.695364,-0.815165,1.02178,-0.0543805,-0.177725,-0.0770195,-0.777284,0.160493,-0.741521,-1.99891,-1.89142,0.479688,-0.798538,-0.903493,-0.424593,-0.51473,1.01341,-0.867649,0,0,0
6,-2.28722,-1.08717,-2.28722,-0.97868,-0.0543805,5.62666,-0.0770195,1.91791,-1.03938,0.398425,0.397638,0.589643,-1.434,1.38397,0.222585,-0.208086,-1.04064,-1.5902,0.17651,1,0,1
7,1.17644,1.26365,1.17644,1.02178,-0.0543805,-0.177725,-0.0770195,1.19228,-0.332109,-1.88147,1.37124,1.41666,-1.434,-0.798538,0.222585,0.00842164,1.85188,-2.1562,0.574807,0,1,0
8,1.08985,1.06775,1.08985,-0.97868,-0.0543805,-0.177725,-0.0770195,-2.12488,2.54603,-0.741521,-1.02531,-1.0644,2.39337,-0.798538,1.34866,0.224929,-0.777687,2.71142,1.04257,0,1,0
9,0.656889,0.675947,0.656889,1.02178,-0.0543805,-0.177725,-0.0770195,0.673975,-1.20019,-0.741521,-1.69934,-1.89142,-0.796104,-0.798538,-0.903493,-0.424593,1.06301,-1.1374,-0.867649,0,0,1


In [19]:
loglrg_model_1 = glm(@formula(Mortgage ~ age + income + members_in_household + loan_accounts), XY_train_df, Binomial(), LogitLink())
loglrg_model_2 = glm(@formula(Pension ~ age + income + members_in_household + loan_accounts), XY_train_df, Binomial(), LogitLink())
loglrg_model_3 = glm(@formula(Savings ~ age + income + members_in_household + loan_accounts), XY_train_df, Binomial(), LogitLink())

Y_pred_lr_1 = GLM.predict(loglrg_model_1, X_test_df)
Y_pred_lr_2 = GLM.predict(loglrg_model_2, X_test_df)
Y_pred_lr_3 = GLM.predict(loglrg_model_3, X_test_df)

Y_pred_lr = hcat(Y_pred_lr_1, Y_pred_lr_2, Y_pred_lr_3)

Y_pred_lr = Y_pred_lr .> 0.5

# compute the accuracy
accuracy_lr_1 = sum(Y_pred_lr[:, 1] .== Y_test[:, 1]) / length(Y_test[:, 1])
accuracy_lr_2 = sum(Y_pred_lr[:, 2] .== Y_test[:, 2]) / length(Y_test[:, 2])
accuracy_lr_3 = sum(Y_pred_lr[:, 3] .== Y_test[:, 3]) / length(Y_test[:, 3])

0.6471655328798186

In [20]:
sum(Y_pred_lr, dims=1)

1×3 Matrix{Int64}:
 248  163  510

In [21]:
# change the type of Y_pred_lr to Int64
Y_pred_lr = map(Int64, Y_pred_lr)
# transfer the prediction matrix to a dataframe
Y_pred_lr_df = DataFrame(Y_pred_lr, [:Savings, :Mortgage, :Pension]);

# Modeling

## Results Calculation Function

In [22]:
# Define a function to compute the results
function results_calculation(decision)
    # calculate core metirc
    revenue = sum((p[j] * s[k]) * Y_test[i, j] * decision[i, j, k] for i in 1:size(Y_test, 1) for j in 1:J for k in 1:K)
    budget = sum(c[k] * decision[i, j, k] for i in 1:size(Y_test, 1) for j in 1:J for k in 1:K)
    profit = - sum((c[k] - p[j] * s[k]) * Y_test[i, j] * decision[i, j, k] for i in 1:size(Y_test, 1) for j in 1:J for k in 1:K)
    customers = sum(decision)
    saving = sum(decision[:, 1, :])
    mortgage = sum(decision[:, 2, :])
    pension = sum(decision[:, 3, :])
    gift = sum(decision[:, :, 1])
    newsletter = sum(decision[:, :, 2])
    seminar = sum(decision[:, :, 3])
    return revenue, budget, profit, customers, saving, mortgage, pension, gift, newsletter, seminar
end

results_calculation (generic function with 1 method)

## Greedy Algorithm

In [23]:
function greedy_model(prediction_model)

    # choose the prediction model we want to use
    if prediction_model == "rf"
        offers = Y_pred_rf_df
    elseif prediction_model == "lr"
        offers = Y_pred_lr_df
    else
        println("Please choose a valid prediction model.")
    end


    # Set up the data
    gsol = zeros(Int64, length(offers[:, 1]), length(products), length(channels[:, 1]))
    budget = 0
    revenue = 0
    noffers = length(offers[:, 1])

    # Ensure the 10% per channel by choosing the most promising per channel
    for c in 1:length(channels[:, 1])
        i = 0
        while i < (noffers / 10)
            # Find a possible offer in this channel for a customer not yet done
            added = false
            for o in 1:noffers
                already = false
                for product in 1:length(products)
                    if sum(gsol[o, product, :]) == 1
                        already = true
                        break
                    end
                end
                if already
                    continue
                end
                
                possible = false
                possibleProduct = nothing
                
                for product in 1:length(products)
                    if offers[o, product] == 1
                        possible = true
                        possibleProduct = product
                        break
                    end
                end

                if !possible
                    continue
                end

                if budget >= B
                    break
                end
                
                gsol[o, possibleProduct, c] = 1
                i += 1
                added = true
                budget += channels[c, 2]

                revenue += channels[c, 3] * productValue[possibleProduct]
                break
            end
            
            if !added
                # println("NOT FEASIBLE")
                break
            end
        end
    end

    # return the solution
    return gsol
end


greedy_model (generic function with 1 method)

In [24]:
# Train the model using random forest as the prediction model
Z_greedy_rf = greedy_model("rf")
revenue_greedy_rf, budget_greedy_rf, profit_greedy_rf, customers_greedy_rf, savings_greedy_rf, mortgage_greedy_rf, pension_greedy_rf, gift_greedy_rf, newsletter_greedy_rf, seminar_greedy_rf  = results_calculation(Z_greedy_rf)

# Train the model using logistics regression as the prediction model
Z_greedy_lr = greedy_model("lr")
revenue_greedy_lr, budget_greedy_lr, profit_greedy_lr, customers_greedy_lr, savings_greedy_lr, mortgage_greedy_lr, pension_greedy_lr, gift_greedy_lr, newsletter_greedy_lr, seminar_greedy_lr  = results_calculation(Z_greedy_lr)

# Create a dataframe to store the results
results_greedy = DataFrame(
    Method = ["Greedy", "Greedy"],
    Model = ["Random Forest", "Logistics Regression"],
    Revenue = [revenue_greedy_rf, revenue_greedy_lr],
    Budget = [budget_greedy_rf, budget_greedy_lr],
    Profit = [profit_greedy_rf, profit_greedy_lr],
    Customers = [customers_greedy_rf, customers_greedy_lr],
    Savings = [savings_greedy_rf, savings_greedy_lr],
    Mortgage = [mortgage_greedy_rf, mortgage_greedy_lr],
    Pension = [pension_greedy_rf, pension_greedy_lr],
    Gift = [gift_greedy_rf, gift_greedy_lr],
    Newsletter = [newsletter_greedy_rf, newsletter_greedy_lr],
    Seminar = [seminar_greedy_rf, seminar_greedy_lr]
)

Row,Method,Model,Revenue,Budget,Profit,Customers,Savings,Mortgage,Pension,Gift,Newsletter,Seminar
,String,String,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,Greedy,Random Forest,7920.0,5365.0,4320.0,284,193,0,91,221,63,0
2,Greedy,Logistics Regression,22660.0,12818.0,15026.0,663,206,112,345,221,221,221


## Sample Average Approximation

In [25]:
# define model
function saa_model(I, J, K, F, X, Y, p, s, c, B)
    # define model
    model = Model(Gurobi.Optimizer)
    set_time_limit_sec(model, 30)

    # define variables
    @variable(model, Z[1:I, 1:J, 1:K], Bin)

    # define the constraints
    @constraint(model, sum(c[k] * sum(Z[i, j, k] for i in 1:I for j in 1:J) for k in 1:K) <= B)
    # @constraint(model, [k=1:K], sum(Z[i, j, k] for i in 1:I for j in 1:J) >= 0.1 * I)
    @constraint(model, [i=1:I, k=1:K], sum(Z[i, j, k] for j in 1:J) <= 1)
    @constraint(model, [i=1:I, j=1:J], sum(Z[i, j, k] for k in 1:K) <= 1)
    for i_1 in 1:I
        for i_2 in 1:I
            @constraint(model, [j=1:J, k=1:K], Z[i_1, j, k] == Z[i_2, j, k])
        end
    end

    # define the objective function
    @objective(model, Min, 1 / I * sum((c[k] - p[j] * s[k]) * Y[i, j] * Z[i, j, k] for i in 1:I for j in 1:J for k in 1:K))

    # solve the model
    optimize!(model)

    # return the decision variables
    return value.(Z)
end

saa_model (generic function with 1 method)

In [54]:
# # build model
# Z_saa = saa_model(I, J, K, F, X_train, Y_train, p, s, c, B);
# # calculate results
# revenue_saa, budget_saa, profit_saa, customers_saa, savings_saa, mortgage_saa, pension_saa, gift_saa, newsletter_saa, seminar_saa  = results_calculation(Z_saa)
# # create a dataframe to store the results
# results_saa = DataFrame(
#     Method = ["Sample Average Approximation"],
#     Model = ["Average"],
#     Revenue = [revenue_saa],
#     Budget = [budget_saa],
#     Profit = [profit_saa],
#     Customers = [customers_saa],
#     Savings = [savings_saa],
#     Mortgage = [mortgage_saa],
#     Pension = [pension_saa],
#     Gift = [gift_saa],
#     Newsletter = [newsletter_saa],
#     Seminar = [seminar_saa]
# )

# SAA is infeasible, so assign 0 to all the results
results_saa = DataFrame(
    Method = ["SAA"],
    Model = ["Sample Average"],
    Revenue = [0],
    Budget = [0],
    Profit = [0],
    Customers = [0],
    Savings = [0],
    Mortgage = [0],
    Pension = [0],
    Gift = [0],
    Newsletter = [0],
    Seminar = [0]
)

Row,Method,Model,Revenue,Budget,Profit,Customers,Savings,Mortgage,Pension,Gift,Newsletter,Seminar
,String,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,SAA,Sample Average,0,0,0,0,0,0,0,0,0,0


## Point Prediction

In [27]:
# buil model for point prediction
function pp_model(J, K, X, Y, p, s, c, B)
    # get number of customers
    I = size(X, 1)

    # define model
    model = Model(Gurobi.Optimizer)

    # define variables
    @variable(model, Z[1:I, 1:J, 1:K], Bin)

    # define the constraints
    @constraint(model, sum(c[k] * sum(Z[i, j, k] for i in 1:I for j in 1:J) for k in 1:K) <= B)
    @constraint(model, [k=1:K], sum(Z[i, j, k] for i in 1:I for j in 1:J) >= 0.1 * I)
    @constraint(model, [i=1:I, k=1:K], sum(Z[i, j, k] for j in 1:J) <= 1)
    @constraint(model, [i=1:I, j=1:J], sum(Z[i, j, k] for k in 1:K) <= 1)

    # define the objective function
    @objective(model, Min, sum((c[k] - p[j] * s[k]) * Y[i, j] * Z[i, j, k] for i in 1:I for j in 1:J for k in 1:K))

    # solve the model
    optimize!(model)

    # return the results
    return value.(Z), objective_value(model)
end

pp_model (generic function with 1 method)

In [28]:
# build model for random forest
Z_pp_rf, objective_pp_rf = pp_model(J, K, X_test, Y_pred_rf, p, s, c, B);
# calculate results
revenue_pp_rf, budget_pp_rf, profit_pp_rf, customers_pp_rf, savings_pp_rf, mortgage_pp_rf, pension_pp_rf, gift_pp_rf, newsletter_pp_rf, seminar_pp_rf  = results_calculation(Z_pp_rf)
# create a dataframe to store the results
results_pp_rf = DataFrame(
    Method = ["Point Prediction"],
    Model = ["Random Forest"],
    Revenue = [revenue_pp_rf],
    Budget = [budget_pp_rf],
    Profit = [profit_pp_rf],
    Customers = [customers_pp_rf],
    Savings = [savings_pp_rf],
    Mortgage = [mortgage_pp_rf],
    Pension = [pension_pp_rf],
    Gift = [gift_pp_rf],
    Newsletter = [newsletter_pp_rf],
    Seminar = [seminar_pp_rf]
)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-17
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 13234 rows, 19845 columns and 79380 nonzeros
Model fingerprint: 0x7c21a224
Variable types: 0 continuous, 19845 integer (19845 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective -9265.000000
Presolve time: 0.04s
Presolved: 13234 rows, 19845 columns, 79380 nonzeros
Variable types: 0 continuous, 19845 integer (19845 binary)
Found heuristic solution: objective -15541.00000

Root relaxation: objective -1.684800e+04, 235 iterations, 0.03 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

Row,Method,Model,Revenue,Budget,Profit,Customers,Savings,Mortgage,Pension,Gift,Newsletter,Seminar
,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Point Prediction,Random Forest,20310.0,14267.0,13807.0,726.0,297.0,91.0,338.0,221.0,221.0,284.0


In [29]:
# build model for logistics regression
Z_pp_lr, objective_pp_lr = pp_model(J, K, X_test, Y_pred_lr, p, s, c, B);
# calculate results
revenue_pp_lr, budget_pp_lr, profit_pp_lr, customers_pp_lr, savings_pp_lr, mortgage_pp_lr, pension_pp_lr, gift_pp_lr, newsletter_pp_lr, seminar_pp_lr  = results_calculation(Z_pp_lr)
# create a dataframe to store the results
results_pp_lr = DataFrame(
    Method = ["Point Prediction"],
    Model = ["Logistics Regression"],
    Revenue = [revenue_pp_lr],
    Budget = [budget_pp_lr],
    Profit = [profit_pp_lr],
    Customers = [customers_pp_lr],
    Savings = [savings_pp_lr],
    Mortgage = [mortgage_pp_lr],
    Pension = [pension_pp_lr],
    Gift = [gift_pp_lr],
    Newsletter = [newsletter_pp_lr],
    Seminar = [seminar_pp_lr]
)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-17
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 13234 rows, 19845 columns and 79380 nonzeros
Model fingerprint: 0x54fcd253
Variable types: 0 continuous, 19845 integer (19845 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective -29413.00000
Presolve time: 0.04s
Presolved: 13234 rows, 19845 columns, 79380 nonzeros
Variable types: 0 continuous, 19845 integer (19845 binary)
Found heuristic solution: objective -46067.00000

Root relaxation: objective -6.621109e+04, 1543 iterations, 0.03 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

Row,Method,Model,Revenue,Budget,Profit,Customers,Savings,Mortgage,Pension,Gift,Newsletter,Seminar
,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Point Prediction,Logistics Regression,52130.0,24985.0,38752.0,1192.0,308.0,219.0,665.0,221.0,221.0,750.0


## Predictive Prescription

In [30]:
@sk_import neighbors: KNeighborsClassifier 

┌ Info: mkl not found, proceeding to installing non-mkl versions of sci-kit learn via Conda
└ @ ScikitLearn.Skcore /Users/zekiyan/.julia/packages/ScikitLearn/sqLdT/src/Skcore.jl:191
┌ Info: Running `conda install -y -c conda-forge 'scikit-learn>=1.2,<1.3'` in root environment
└ @ Conda /Users/zekiyan/.julia/packages/Conda/sDjAP/src/Conda.jl:181
Error while loading conda entry point: conda-libmamba-solver (dlopen(/Users/zekiyan/.julia/conda/3/aarch64/lib/python3.10/site-packages/libmambapy/bindings.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libarchive.19.dylib
  Referenced from: <494304CB-2F24-3FF8-B4AE-F4E6B16BDC26> /Users/zekiyan/.julia/conda/3/aarch64/lib/libmamba.2.0.0.dylib
  Reason: tried: '/Users/zekiyan/.julia/conda/3/aarch64/lib/libarchive.19.dylib' (no such file), '/Users/zekiyan/.julia/conda/3/aarch64/lib/python3.10/site-packages/libmambapy/../../../libarchive.19.dylib' (no such file), '/Users/zekiyan/.julia/conda/3/aarch64/lib/python3.10/site-packages/libmamb

Solving environment: ...working... done

# All requested packages already installed.



PyObject <class 'sklearn.neighbors._classification.KNeighborsClassifier'>

In [31]:
knn_models = []
X_train = Matrix(X_train)
for k in 1:2:15
    knn = KNeighborsClassifier(k)
    ScikitLearn.fit!(knn, X_train, Y_train)
    push!(knn_models, knn)
end

In [32]:
# create the model
function knn_model(lnr, N, J, K, X, Y, p, s, c, B)
    # X is the test set
    # Y is the train set
    # N is the number of neighbors

    # get neighbors
    Y_neighbors = lnr.kneighbors(X, return_distance=false) 

    # get number of customers
    I = size(X, 1)

    # define model
    model = Model(Gurobi.Optimizer)

    # define variables
    @variable(model, Z[1:I, 1:J, 1:K], Bin)

    # define the constraints
    @constraint(model, sum(c[k] * sum(Z[i, j, k] for i in 1:I for j in 1:J) for k in 1:K) <= B)
    @constraint(model, [k=1:K], sum(Z[i, j, k] for i in 1:I for j in 1:J) >= 0.1 * I)
    @constraint(model, [i=1:I, k=1:K], sum(Z[i, j, k] for j in 1:J) <= 1)
    @constraint(model, [i=1:I, j=1:J], sum(Z[i, j, k] for k in 1:K) <= 1)

    # define the objective function
    @objective(model, Min, sum((c[k] - p[j] * s[k]) * sum(Y[Y_neighbors[i, n] + 1, j] for n in 1:N) / N * Z[i, j, k] for i in 1:I for j in 1:J for k in 1:K))

    # solve the model
    optimize!(model)

    # return the results
    return value.(Z), objective_value(model)
end

knn_model (generic function with 1 method)

In [33]:
# build model
Z_pres_knn, objective_pres_knn = knn_model(knn_models[8], 15, J, K, X_test, Y_train, p, s, c, B);
# calculate results
revenue_pres_knn, budget_pres_knn, profit_pres_knn, customers_pres_knn, savings_pres_knn, mortgage_pres_knn, pension_pres_knn, gift_pres_knn, newsletter_pres_knn, seminar_pres_knn  = results_calculation(Z_pres_knn)
# create a dataframe to store the results
results_pres_knn = DataFrame(
    Method = ["Prescriptive"],
    Model = ["KNN"],
    Revenue = [revenue_pres_knn],
    Budget = [budget_pres_knn],
    Profit = [profit_pres_knn],
    Customers = [customers_pres_knn],
    Savings = [savings_pres_knn],
    Mortgage = [mortgage_pres_knn],
    Pension = [pension_pres_knn],
    Gift = [gift_pres_knn],
    Newsletter = [newsletter_pres_knn],
    Seminar = [seminar_pres_knn]
)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-17
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 13234 rows, 19845 columns and 79380 nonzeros
Model fingerprint: 0x80d9a646
Variable types: 0 continuous, 19845 integer (19845 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Found heuristic solution: objective -10319.00000
Presolve time: 0.05s
Presolved: 13234 rows, 19845 columns, 79380 nonzeros
Variable types: 0 continuous, 19845 integer (19845 binary)
Found heuristic solution: objective -11105.66667

Root relaxation: objective -4.752531e+04, 1488 iterations, 0.04 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

Row,Method,Model,Revenue,Budget,Profit,Customers,Savings,Mortgage,Pension,Gift,Newsletter,Seminar
,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Prescriptive,KNN,53180.0,25000.0,41435.0,1193.0,0.0,60.0,1133.0,221.0,222.0,750.0


## Kernel Methods

In [49]:
function kernel(x_data, x_new, bandwidth)
    x = (x_data - x_new) / bandwidth 
    return Int(sqrt(x' * x) <= 1)
end

kernel (generic function with 1 method)

In [50]:
# Set hyperparameter
h_N = 4

# test result
x = [1, 2, 3]
Int(sqrt(x' * x) <= 1)
X_to_test = X_test[1, :]
sum(Y_train[i, :] * kernel(X_train[i,:], X_to_test, h_N) for i in 1:size(X_train, 1)) / sum(kernel(X_train[i,:], X_to_test, h_N) for i in 1:size(X_train, 1))

3-element Vector{Float64}:
 0.11972437553832903
 0.3970714900947459
 0.4160206718346253

In [51]:
# create the model
function kernel_methods_model(J, K, X, Y, p, s, c, B, X_train)
    # X is the test set
    # Y is the train set
    # N is the number of neighbors

    # get number of customers
    I = size(X, 1)

    # define model
    model = Model(Gurobi.Optimizer)

    # set time limit
    set_time_limit_sec(model, 30)

    # define variables
    @variable(model, Z[1:I, 1:J, 1:K], Bin)

    # define the constraints
    @constraint(model, sum(c[k] * sum(Z[i, j, k] for i in 1:I for j in 1:J) for k in 1:K) <= B)
    @constraint(model, [k=1:K], sum(Z[i, j, k] for i in 1:I for j in 1:J) >= 0.1 * I)
    @constraint(model, [i=1:I, k=1:K], sum(Z[i, j, k] for j in 1:J) <= 1)
    @constraint(model, [i=1:I, j=1:J], sum(Z[i, j, k] for k in 1:K) <= 1)

    # define the objective function
    @objective(model, Min, sum((c[k] - p[j] * s[k]) * sum(sum(Y[g, :] * kernel(X_train[g,:], X[i, :], h_N) for g in 1:size(X_train, 1)) / sum(kernel(X_train[g,:], X[i, :], h_N) for g in 1:size(X_train, 1)) for i in 1:I) * Z[i, j, k] for i in 1:I for j in 1:J for k in 1:K))

    # solve the model
    optimize!(model)

    # return the results
    return value.(Z), objective_value(model)
end

kernel_methods_model (generic function with 1 method)

In [39]:
# Z_km, objective_km = kernel_methods_model(J, K, X_test, Y_train, p, s, c, B, X_train)

# Final Score Comparison

In [55]:
# combine all the results
results = vcat(results_greedy, results_saa, results_pp_rf, results_pp_lr, results_pres_knn)

Row,Method,Model,Revenue,Budget,Profit,Customers,Savings,Mortgage,Pension,Gift,Newsletter,Seminar
,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Greedy,Random Forest,7920.0,5365.0,4320.0,284.0,193.0,0.0,91.0,221.0,63.0,0.0
2,Greedy,Logistics Regression,22660.0,12818.0,15026.0,663.0,206.0,112.0,345.0,221.0,221.0,221.0
3,SAA,Sample Average,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Point Prediction,Random Forest,20310.0,14267.0,13807.0,726.0,297.0,91.0,338.0,221.0,221.0,284.0
5,Point Prediction,Logistics Regression,52130.0,24985.0,38752.0,1192.0,308.0,219.0,665.0,221.0,221.0,750.0
6,Prescriptive,KNN,53180.0,25000.0,41435.0,1193.0,0.0,60.0,1133.0,221.0,222.0,750.0
